# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 
 The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
#Create a Keyspace 
try:
    session.execute("""
                    CREATE KEYSPACE IF NOT EXISTS sparkify
                    WITH REPLICATION =
                    {'class': 'SimpleStrategy', 'replication_factor': 1}
                    """
                   )
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
#Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

# Model data with the queries that we want  
### 1. artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


To get the query that we want we must create a primary key that will consist of a partition key: sessionId, and a clustering key: itemInSession.   

SessionId is the partition key and itemInSession is the clustering key because the data is split by the sessionId and for each sessionId, the partition is seperated by a itemInSession.

The columns we will need are sessionId, itemInSession, artist, song, length.  

In [8]:
query = "CREATE TABLE IF NOT EXISTS session_items "
query = query + "( sessionId int, itemInSession int, artist varchar, song varchar, length float, PRIMARY KEY(sessionId, itemInSession))"

try: 
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##Assign the INSERT statements into the `query` variable
        query = "INSERT INTO session_items ( sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        ## Assign column element for each column in the INSERT statement.
        session.execute(query, ( int(line[8]), int(line[3]), line[0], line[10], float(line[5])) )

*Do a SELECT to verify that the data have been inserted into each table*

In [10]:
query = """
        SELECT artist,
               song,
               length
        FROM session_items
        WHERE sessionId = 338 AND itemInSession = 4
"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row)

Row(artist='Faithless', song='50', length=495.30731201171875)


### 2. name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

To get the query that we want we must create a primary key that will consist of a partition key: (userId, sessionId), and a clustering key: itemInSession.   

UserId and SessionId is the partition key and itemInSession is the clustering key because for each unqiue UserId and SessionId we see a unique itemInSession.

The columns we will need are UserId, sessionId, itemInSession, artist, song, firstName, lastName.

In [11]:
query = "CREATE TABLE IF NOT EXISTS session_users "
query = query + "(userId int, sessionId int, itemInSession int, artist varchar, song varchar, firstName varchar, lastName varchar, PRIMARY KEY((userId, sessionId), itemInSession))"
query = query + " WITH CLUSTERING ORDER BY (itemInSession ASC)"
try: 
    session.execute(query)
except Exception as e:
    print(e)

                    

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO session_users (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        ## Assign column element for each column in the INSERT statement.
        session.execute(query, (int(line[-1]), int(line[8]), int(line[3]), line[0], line[10], line[1], line[4]) )

*Do a SELECT to verify that the data have been inserted into each table*

In [13]:
query = """
         SELECT  artist,
                 song,
                 firstName,
                 lastName
         FROM session_users
         WHERE userId = 10 AND sessionId = 182
            """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(artist='Down To The Bone', song='10', firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='10', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='10', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='10', firstname='Sylvie', lastname='Cruz')


### 3. Every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

To get the query that we want we must create a primary key that will consist of a partition key: song, and a clustering key: userId.  

song is the partition key and userId is the clustering key because we want the song and for each song there are many different userIds.

The columns we will need are song, userId, firstName, lastName.

In [14]:
query = "CREATE TABLE IF NOT EXISTS session_songs "
query = query + "( song varchar, userId int, firstName varchar, lastName varchar, PRIMARY KEY(song,userId))"

try: 
    session.execute(query)
except Exception as e:
    print(e)

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##Assign the INSERT statements into the `query` variable
        query = "INSERT INTO session_songs (song, userId, firstName, lastName)"
        query = query + "VALUES(%s, %s, %s, %s)"
        ## Assign column element for each column in the INSERT statement.
        session.execute(query, ( line[-2], int(line[-1]), line[1], line[4]) )

*Do a SELECT to verify that the data have been inserted into each table*

In [17]:
query = """
         SELECT  firstName,
                 lastName
         FROM session_songs
         WHERE song = 'All Hands Against His Own'
         """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [18]:
session.execute("DROP TABLE IF EXISTS sparkify.session_items")
session.execute("DROP TABLE IF EXISTS sparkify.session_users")
session.execute("DROP TABLE IF EXISTS sparkify.session_songs")

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()